In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("scater")
install.packages('Seurat')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.2 (2023-10-31)

Installing package(s) 'BiocVersion', 'scater'

also installing the dependencies ‘zlibbioc’, ‘bitops’, ‘XVector’, ‘sparseMatrixStats’, ‘RCurl’, ‘GenomeInfoDbData’, ‘abind’, ‘lambda.r’, ‘futile.options’, ‘sitmo’, ‘GenomicRanges’, ‘DelayedMatrixStats’, ‘Biobase’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘SparseArray’, ‘matrixStats’, ‘RcppHNSW’, ‘ScaledMatrix’, ‘irlba’, ‘rsvd’, ‘futile.logger’, ‘snow’, ‘BH’, ‘beeswarm’, ‘vipor’, ‘gridExtra’, ‘FNN’, ‘RcppAnnoy’, ‘RcppProgress’, ‘dqrng’, ‘Cairo’, ‘png’, ‘SingleCellExperiment’, ‘scuttle’, ‘BiocGenerics’, ‘S4Vectors’, ‘SummarizedExperiment’, ‘DelayedArray’, ‘MatrixGenerics’, ‘beachmat’, ‘BiocNeighbors’, ‘

In [ ]:
if (!requireNamespace("remotes", quietly = TRUE)) {
  install.packages("remotes")
}
remotes::install_github("mojaveazure/seurat-disk")

In [ ]:
library(scater)
library(Seurat)
library(SeuratDisk)
# library(SeuratData)
library(patchwork)

In [2]:

a08 <- Connect(filename = "/home/tchari/counts/allen_bivi/loom/processed_allen_A08_raw.loom", mode = "r")

Class: loom
Filename: /home/tchari/counts/allen_bivi/loom/processed_allen_A08_raw.loom
Access type: H5F_ACC_RDONLY
Attributes: last_modified
Listing:
       name    obj_type dataset.dims dataset.type_class
      attrs   H5I_GROUP         <NA>               <NA>
  col_attrs   H5I_GROUP         <NA>               <NA>
 col_graphs   H5I_GROUP         <NA>               <NA>
     layers   H5I_GROUP         <NA>               <NA>
     matrix H5I_DATASET 5904 x 32285          H5T_FLOAT
  row_attrs   H5I_GROUP         <NA>               <NA>
 row_graphs   H5I_GROUP         <NA>               <NA>

In [18]:
# gene list
n_genes <- a08[['row_attrs']][['gene_name']][['dims']]
gns <- a08[['row_attrs']][['gene_name']][1:n_genes]

# cell ID list
n_cells <- a08[['col_attrs']][['barcode']][['dims']]
cellids <- a08[['col_attrs']][['barcode']][1:n_cells]

# get raw counts matrix
raw.cnts <- t(a08[["layers/unspliced"]][,])
colnames(raw.cnts) <- cellids
# rownames(raw.cnts) <- gns

In [19]:
metadata <- data.frame(
    cellID = cellids
    )
rownames(metadata) <- cellids

In [20]:
#as.data.frame(t(a08[["layers/unspliced"]][,] ))


s_obj <- CreateSeuratObject(counts = raw.cnts,
                            project = "fromLoom",
                            assay = "URNA",
                            meta.data = metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [23]:
sraw.cnts <- t(a08[["layers/spliced"]][,])
colnames(sraw.cnts) <- cellids

s_assay <- CreateAssay5Object(counts = sraw.cnts)

s_obj[["SRNA"]] <- s_assay

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [24]:
Assays(s_obj)

[1] "URNA" "SRNA"

In [25]:
a08$close_all()

In [29]:
DefaultAssay(s_obj) <- 'URNA'
s_obj <- NormalizeData(s_obj) 
s_obj <- FindVariableFeatures(s_obj) 
s_obj <- ScaleData(s_obj) 
s_obj <- RunPCA(s_obj,reduction.name = 'upca')

DefaultAssay(s_obj) <- 'SRNA'
s_obj <- NormalizeData(s_obj) 
s_obj <- FindVariableFeatures(s_obj)
s_obj <- ScaleData(s_obj)
s_obj <- RunPCA(s_obj,reduction.name = 'spca')


Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  Feature18712, Feature4557, Feature1179, Feature25330, Feature18508, Feature15353, Feature29169, Feature6087, Feature10433, Feature24016 
	   Feature23056, Feature2070, Feature31029, Feature16451, Feature697, Feature7113, Feature2293, Feature29160, Feature7362, Feature12693 
	   Feature23338, Feature7126, Feature21608, Feature21840, Feature12343, Feature11406, Feature2493, Feature27801, Feature13683, Feature30577 
Negative:  Feature2895, Feature7340, Feature17730, Feature29597, Feature23503, Feature16409, Feature18860, Feature22149, Feature5807, Feature22797 
	   Feature25791, Feature4598, Feature6197, Feature23146, Feature4117, Feature28198, Feature23489, Feature772, Feature28064, Feature14487 
	   Feature29246, Feature9908, Feature22708, Feature21288, Feature4866, Feature23078, Feature18251, Feature8530, Feature10584, Feature20988 
PC_ 2 
Positive:  Feature22821,

In [30]:
s_obj <- FindMultiModalNeighbors(
  s_obj, reduction.list = list("upca", "spca"), 
  dims.list = list(1:30, 1:18), modality.weight.name = "RNA.weight"
)

Calculating cell-specific modality weights

Finding 20 nearest neighbors for each modality.

Calculating kernel bandwidths

Warning message in FindMultiModalNeighbors(s_obj, reduction.list = list("upca", :
“The number of provided modality.weight.name is not equal to the number of modalities. URNA.weight SRNA.weight are used to store the modality weights”
Finding multimodal neighbors

Constructing multimodal KNN graph

Constructing multimodal SNN graph



In [31]:
s_obj[["wknn"]]

A Graph object containing 5904 cells

In [32]:
#MAybe just run Leiden here + save results